## Example

### Definicion de funciones necesarias para generacion malla

In [77]:
%config IPCompleter.greedy=True
import numpy as np
import os
import bempp.api
from mesh_converter import *
from force_calculation import *

### Seleccion de proteina a simular, contenidas en carpetas pqr_files

Se determina las ubicaciones de las cargas ademas de la magnitud de estas

In [78]:
#grid, q, x_q = pqrtomesh(directory='C:\\Users\\ian\Desktop\\forces_calculation',protein='sphere',forcefield='nosymmetric',density=6.0,probe_radius=1.4)

In [79]:
directory = 'C:\\Users\\ian\Desktop\\forces_calculation' #Windows
protein = 'arg'
forcefield = 'amber'
dir_prot = directory+'\\pqr_files\\'+protein #Windows
density = 0.2
pf = protein +'_' + forcefield
pfd = protein +'_' + forcefield + '_' +'d'+str(density)[::2]
grid = import_msms_mesh('{}/{}.face'.format(dir_prot,pfd),'{}/{}.vert'.format(dir_prot,pfd))
#Parametros del medio
q, x_q = np.array([]), np.empty((0,3))
ep_in = 4.
ep_ex = 80.
k = 0.125

#Leer cargas y coordenadas desde archivo .pqr
molecule_file = open('{}/{}.pqr'.format(dir_prot,pf), 'r').read().split('\n')
for line in molecule_file:
    line = line.split()
    if len(line)==0 or line[0]!='ATOM': continue
    q = np.append( q, float(line[8]))
    x_q = np.vstack(( x_q, np.array(line[5:8]).astype(float) ))

### Grafico de archivo mesh (Opcional)

In [80]:
bempp.api.PLOT_BACKEND = "gmsh"
grid.plot()

### Right-hand side

With that, we can compute the potential due to the charges on the boundary, required for the right-hand side

In [81]:
# Function to calculate the potential by charges at boundary
ep_in = 4.
ep_ex = 80.
k = 0.125

dirichl_space = bempp.api.function_space(grid, "DP", 0)
neumann_space = bempp.api.function_space(grid, "DP", 0)

@bempp.api.real_callable
def charges_fun(x, n, domain_index, result):
    global q, x_q, ep_in
    suma = 0
    for k in range(len(q)):
        suma = suma + q[k]/(np.linalg.norm(x-x_q[k]))
    result[:] = suma/(4*np.pi*ep_in)
    
charged_grid_fun = bempp.api.GridFunction(dirichl_space, fun=charges_fun)

rhs = np.concatenate([charged_grid_fun.coefficients, 
                      np.zeros(neumann_space.global_dof_count)])


C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


### Operators and Matrix

Next, we generate the $2\times 2$ block matrix with the single and double layer operators of the Laplace and Yukawa kernels 

In [82]:
# Define Operators
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
identity = sparse.identity(dirichl_space, dirichl_space, dirichl_space)
slp_in   = laplace.single_layer(neumann_space, dirichl_space, dirichl_space)
dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dirichl_space)
slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dirichl_space, k)
dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dirichl_space, k)

# Matrix Assembly
blocked = bempp.api.BlockedOperator(2, 2)
blocked[0, 0] = 0.5*identity + dlp_in
blocked[0, 1] = -slp_in
blocked[1, 0] = 0.5*identity - dlp_out
blocked[1, 1] = (ep_in/ep_ex)*slp_out
op_discrete = blocked.strong_form()

### Solver

We now use `gmres` from `scipy` to solve the system. 

In [83]:
import inspect
from scipy.sparse.linalg import gmres

array_it, array_frame, it_count = np.array([]), np.array([]), 0
def iteration_counter(x):
        global array_it, array_frame, it_count
        it_count += 1
        frame = inspect.currentframe().f_back
        array_it = np.append(array_it, it_count)
        array_frame = np.append(array_frame, frame.f_locals["resid"])
        #print "It: {0} Error {1:.2E}".format(it_count, frame.f_locals["resid"])        

x, info = gmres(op_discrete, rhs, callback=iteration_counter, tol=1e-3, maxiter=1000, restart = 2000)

print("The linear system was solved in {0} iterations".format(it_count))

The linear system was solved in 36 iterations


The two following `GridFunction` calls store the calculated boundary potential data (separated by $\phi$ and $\frac{\partial \phi}{\partial n}$) for visualization purposes. 

In [84]:
solution_dirichl = bempp.api.GridFunction(dirichl_space, 
                                          coefficients=x[:dirichl_space.global_dof_count])
solution_neumann = bempp.api.GridFunction(neumann_space, 
                                          coefficients=x[dirichl_space.global_dof_count:])
solution_dirichl.plot()
#solution_neumann.plot()

### Energy calculaton

To compute $\phi_\text{reac}$ at the atoms locations, we use `operators.potential` to then multiply by the charge and add to compute $\Delta G_\text{solv}$. The `332.064` term is just a unit conversion constant.

In [85]:
slp_q = bempp.api.operators.potential.laplace.single_layer(neumann_space, x_q.transpose())
dlp_q = bempp.api.operators.potential.laplace.double_layer(dirichl_space, x_q.transpose())
phi_q = slp_q*solution_neumann - dlp_q*solution_dirichl

h=0.001
F_qf,_ = fixedcharge_forces(x_q,q,h,neumann_space,dirichl_space,solution_neumann,solution_dirichl)
f_db,f_ib = boundary_forces(solution_neumann,solution_dirichl,grid,k,ep_ex,ep_in)
  
print("Total reaction force: {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(F_qf[0],F_qf[1],F_qf[2]))
print("Total dielectric boundary force: {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(f_db[0],f_db[1],f_db[2]))
print("Total ionic boundary force: {:5.3e} {:5.3e} {:5.3e} [kJ/molA]".format(f_ib[0],f_ib[1],f_ib[2]))
# total dissolution energy applying constant to get units [kcal/mol]
total_energy = 4.184*2*np.pi*332.064*np.sum(q*phi_q).real
print("Total solvation energy: {:7.2f} [kcal/Mol]".format(total_energy))

Total reaction force:    34.3747  -10.9690   11.2223 [kJ/molA]
Total dielectric boundary force:   -35.3895   10.0886  -13.0698 [kJ/molA]
Total ionic boundary force: -1.517e-02 -2.000e-03 -1.387e-02 [kJ/molA]
Total solvation energy: -106.22 [kcal/Mol]


## References
[1] Yoon, B. J., & Lenhoff, A. M. (1990). A boundary element method for molecular electrostatics with electrolyte effects. *Journal of Computational Chemistry*, 11(9), 1080-1086.

[2] Dolinsky, T. J., Nielsen, J. E., McCammon, J. A., & Baker, N. A. (2004). PDB2PQR: an automated pipeline for the setup of Poisson–Boltzmann electrostatics calculations. *Nucleic acids research*, 32(suppl_2), W665-W667.

[3] Sanner, M. F., Olson, A. J., & Spehner, J. C. (1995, September). Fast and robust computation of molecular surfaces. In *Proceedings of the eleventh annual symposium on Computational geometry* (pp. 406-407). ACM.

[4] Sergio Decherchi and Walter Rocchia. A general and Robust Ray-Casting-Based
Algorithm for Triangulating Surfaces at the Nanoscale. PLOS ONE, 8(4):1–15, 2013.

[5] Timo Betcke and Matthew W. Scroggs. Bempp-cl: A fast python based just-in-time
compiling boundary element library. Journal of Open Source Software, 6(59):2879,
2021.

[6] S. Decherchi, A. Spitaleri, J. Stone and W. Rocchia, "NanoShaper-VMD interface: computing and visualizing surfaces, pockets and channels in
molecular systems". Bioinformatics, in press, 2018.

In [86]:
from apbs_tools import *
pqr_path = 'C:\\Users\\ian\\Desktop\\forces_calculation\\pqr_files\\1lyz\\1lyz_amber.pqr'
apbs_dir = 'C:\\Users\\ian\\Desktop\\forces_calculation\\APBS'
protein = '1lyz_amber'

In [87]:
#apbs_simulation(protein,pqr_path,apbs_dir,150,70,'spl4','yes','no')
#apbs_simulation(protein,pqr_path,apbs_dir,200,70,'spl4','yes','no')
#apbs_simulation(protein,pqr_path,apbs_dir,250,70,'spl4','yes','no')
#apbs_simulation(protein,pqr_path,apbs_dir,300,70,'spl4','yes','no')
#apbs_simulation(protein,pqr_path,apbs_dir,350,70,'spl4','yes','no')

In [88]:
#apbs_simulation(protein,pqr_path,apbs_dir,150,70,'mol','no','no')
#apbs_simulation(protein,pqr_path,apbs_dir,200,70,'mol','no','no')
#apbs_simulation(protein,pqr_path,apbs_dir,250,70,'mol','no','no')
#apbs_simulation(protein,pqr_path,apbs_dir,300,70,'mol','no','no')
#apbs_simulation(protein,pqr_path,apbs_dir,350,70,'mol','no','no')

In [1]:
from PB_solver import Molecule
import bempp.api
bempp.api.PLOT_BACKEND = "gmsh"




In [2]:
sphere_nosymmetric_6 = Molecule('sphere','nosymmetric',6.0,None)
sphere_nosymmetric_8 = Molecule('sphere','nosymmetric',8.0,None)
sphere_nosymmetric_10 = Molecule('sphere','nosymmetric',10.0,None)

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [8]:
sphere_nosymmetric_12 = Molecule('sphere','nosymmetric',12.0,None)

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [9]:
sphere_symmetric_3 = Molecule('sphere','symmetric',3.0,None)
sphere_symmetric_5 = Molecule('sphere','symmetric',5.0,None)
sphere_symmetric_8 = Molecule('sphere','symmetric',8.0,None)
sphere_symmetric_10 = Molecule('sphere','symmetric',10.0,None)

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [12]:
arg_d02 = Molecule('arg','amber',0.2,'yes')
arg_d04 = Molecule('arg','amber',0.4,'yes')
arg_d08 = Molecule('arg','amber',0.8,'yes')
arg_d16 = Molecule('arg','amber',1.6,'yes')

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [58]:
arg_d16.grid.number_of_elements

6520

In [2]:
lyz_d04 = Molecule('1lyz','amber',0.4,'yes')
lyz_d06 = Molecule('1lyz','amber',0.6,'yes')
lyz_d08 = Molecule('1lyz','amber',0.8,'yes')

C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [28]:
#lyz_d08.solv_energy
#lyz_d08.F_qf
#lyz_d08.F_db
#lyz_d08.F_ib
lyz_d08.grid.number_of_elements

10156